# IDAES Skeleton Unit Model

This notebook demonstrates usage of the IDAES Skeleton Unit Model, which provides a generic "bare bones" unit for user-defined models and custom variable and constraint sets. To allow maximum versatility, this unit may be defined as a surrogate model or a custom equation-oriented model. Users must add ports and variables that match connected models, and this is facilitated through a provided method to add port-variable sets.

For users who wish to train surrogates with IDAES tools and insert obtained models into a flowsheet, see more detailed information on [IDAES Surrogate Tools](https://idaes-pse.readthedocs.io/en/stable/explanations/modeling_extensions/surrogate/index.html).

# 1. Motivation

In many cases, a specific application requires a unique unit operation that does not exist in the IDAES repository. Custom user models may source from external scripts, import surrogate equations or use first-principles calculations. However, IDAES flowsheets adhere to a standardized modeling hierarchy and simple Pyomo models do not always follow these conventions. Additionally, simple flowsheet submodels often require integration with other IDAES unit models which requires consistency between corresponding port variables, stream properties and physical unit sets, as well as proper usage of `ControlVolume` blocks.

The IDAES `SkeletonUnitModel` allows custom creation of user models blocks that do not require `ControlVolume` blocks, and enabling connection with standard IDAES unit models that do contain `ControlVolume` blocks. To motivate the usefulness and versatility of this tool, we will consider a simple pervaporation unit. The custom model does not require rigourous thermodynamic calculations contained in adjacent unit models, and using a Skeleton model allows definition of only required variables and constraints. The new block does require state variable connections for the inlet and outlet streams. We will demonstrate this scenario below to highlight the usage and benefits of the Skeleton model.

# 2. Example - Pervaporation

Pervaporation is a low-energy separation process, and is particularly advantageous over distillation for azeotropic solutions or aqueous mixtures of heavy alcohols. Ethylene glycol is more environmentally friendly than typical chloride- and bromide-based dessicants, and is a common choice for commericial recovery of water from flue gas via liquid spray columns. Due to ethylene glycol's high boiling point, diffusion-based water recovery is economically favorable compared to distillation-based processes. The following example and flux correlation are taken from the literature source below:

Jennifer Runhong Du, Amit Chakma, X. Feng, Dehydration of ethylene glycol by pervaporation using poly(N,N-dimethylaminoethyl methacrylate)/polysulfone composite membranes, Separation and Purification Technology, Volume 64, Issue 1, 2008, Pages 63-70, ISSN 1383-5866, https://doi.org/10.1016/j.seppur.2008.08.004.

The diagram below is taken from the literature, showing an inlet aqueous glycol feed circulated through a feed tank-membrane-feed tank recycle loop while permeate is continuously extracted by the membrane. To demonstrate the usefulness of the Skeleton model, we will model this system as a Mixer and custom Pervaporation unit define the flux as an empirical custom mass balance term rather than requiring rigorous diffusion calculations. We will also circumvent the need for a vapor phase and VLE calculations by manually calculating the duty to condense and collect permeate vapor, and use correlations for steady-state fluxes to avoid a recycle requiring tear calculations.

![](pervaporation.png)

## 2.1 Pyomo and IDAES Imports
We will begin with relevant imports. We will need basic Pyomo and IDAES components:

In [1]:
import pytest
from pyomo.environ import (check_optimal_termination,
                           ConcreteModel,
                           Constraint,
                           Expression,
                           Objective,
                           maximize,
                           Var,
                           Set,
                           TransformationFactory,
                           value,
                           exp,
                           units as pyunits)
from pyomo.network import Arc
from idaes.core import FlowsheetBlock
from idaes.models.unit_models import SkeletonUnitModel, Mixer, Heater
from idaes.core.util.model_statistics import degrees_of_freedom
from idaes.core.util.initialization import propagate_state
from idaes.core.solvers import get_solver
from pyomo.util.check_units import assert_units_consistent

# import thermophysical properties
import eg_h2o_ideal as thermo_props
from idaes.models.properties.modular_properties.base.generic_property import (
        GenericParameterBlock)
from idaes.core.util.constants import Constants

## 2.2 Build Flowsheet

We will build a simple model manually defining state variables relations entering and exiting the pervaporation unit. As shown below, we may define our pre-separation mixer as usual:

In [2]:
# build the flowsheet
m = ConcreteModel()
m.fs = FlowsheetBlock(default={"dynamic": False})

m.fs.thermo_params = GenericParameterBlock(default=thermo_props.config_dict)

m.fs.M101 = Mixer(default={"property_package": m.fs.thermo_params,
                           "inlet_list": ["water_feed", "glycol_feed"]})

## 2.2 Defining Skeleton Model and Connections

Now that our flowsheet exists, we can manually define variables, units, constraints and ports for our custom pervaporation unit model. By using a Skeleton model, we avoid rigorous mass and energy balances and phase equilibrium which impact model tractability. Instead, we define state variable relations as below - note that we include the fluxes as outlet flow terms. In this model, the variables specify an `FpcTP` system where molar flow of each component, temperature and pressure are selected as state variables:

In [3]:
# define Skeleton model for pervaporation unit
m.fs.pervap = SkeletonUnitModel(default={"dynamic": False})
m.fs.pervap.comp_list = Set(initialize=["water", "ethylene_glycol"])
m.fs.pervap.phase_list = Set(initialize=["Liq"])

# input vars for skeleton
# m.fs.time is a pre-initialized Set belonging to the FlowsheetBlock; for dynamic=False, time=[0]
m.fs.pervap.flow_in = Var(m.fs.time, m.fs.pervap.phase_list, m.fs.pervap.comp_list,
                                     initialize=1.0, units=pyunits.mol/pyunits.s)
m.fs.pervap.temperature_in = Var(m.fs.time, initialize=298.15, units=pyunits.K)
m.fs.pervap.pressure_in = Var(m.fs.time, initialize=101e3, units=pyunits.Pa)

# output vars for skeleton
m.fs.pervap.perm_flow = Var(m.fs.time, m.fs.pervap.phase_list, m.fs.pervap.comp_list,
                                      initialize=1.0, units=pyunits.mol/pyunits.s)
m.fs.pervap.ret_flow = Var(m.fs.time, m.fs.pervap.phase_list, m.fs.pervap.comp_list,
                                      initialize=1.0, units=pyunits.mol/pyunits.s)
m.fs.pervap.temperature_out = Var(m.fs.time, initialize=298.15, units=pyunits.K)
m.fs.pervap.pressure_out = Var(m.fs.time, initialize=101e3, units=pyunits.Pa)
m.fs.pervap.vacuum = Var(m.fs.time, initialize=1.3e3, units=pyunits.Pa)

# dictionaries relating state properties to custom variables
inlet_dict = {"flow_mol_phase_comp": m.fs.pervap.flow_in,
              "temperature": m.fs.pervap.temperature_in,
              "pressure": m.fs.pervap.pressure_in}
retentate_dict = {"flow_mol_phase_comp": m.fs.pervap.ret_flow,
                  "temperature": m.fs.pervap.temperature_out,
                  "pressure": m.fs.pervap.pressure_out}
permeate_dict = {"flow_mol_phase_comp": m.fs.pervap.perm_flow,
                 "temperature": m.fs.pervap.temperature_out,
                 "pressure": m.fs.pervap.vacuum}

m.fs.pervap.add_ports(name="inlet", member_dict=inlet_dict)
m.fs.pervap.add_ports(name="retentate", member_dict=retentate_dict)
m.fs.pervap.add_ports(name="permeate", member_dict=permeate_dict)

# internal vars for skeleton
energy_activation_dict = {(0, 'Liq', 'water'): 51e3, (0, 'Liq', 'ethylene_glycol'): 53e3}
m.fs.pervap.energy_activation = Var(m.fs.time, m.fs.pervap.phase_list, m.fs.pervap.comp_list,
                                    initialize=energy_activation_dict, units=pyunits.J/pyunits.mol)
m.fs.pervap.energy_activation.fix()

permeance_dict = {(0, 'Liq', 'water'): 5611320, (0, 'Liq', 'ethylene_glycol'): 22358.88}  # calculated from literature data
m.fs.pervap.permeance = Var(m.fs.time, m.fs.pervap.phase_list, m.fs.pervap.comp_list,
                            initialize=permeance_dict, units=pyunits.mol/pyunits.s/pyunits.m**2)
m.fs.pervap.permeance.fix()

m.fs.pervap.area = Var(m.fs.time, initialize=6, units = pyunits.m**2)
m.fs.pervap.area.fix()

cond_heat_dict = {(0, 'Liq', 'water'): 40.660e3, (0, 'Liq', 'ethylene_glycol'): 56.9e3}
m.fs.pervap.cond_heat = Var(m.fs.time, m.fs.pervap.phase_list, m.fs.pervap.comp_list,
                            initialize=cond_heat_dict, units=pyunits.J/pyunits.mol)
m.fs.pervap.cond_heat.fix()
m.fs.pervap.cond_duty = Var(m.fs.time, initialize=1, units=pyunits.J/pyunits.s)  # we will calculate this later

Let's define our surrogate equations for flux and permeance, and link them to the port variables. Users can use this structure to write custom relations between inlet and outlet streams; for example, here we define the outlet flow of the pervaporation unit as a sum of the inlet flow and calculated recovery fluxes. By defining model constraints in lieu of rigorous mass balances, we add the flux as a custom mass balance term via an empirical correlation and calculate only the condensation duty rather than implementing full energy balance calculations:

In [4]:
# Surrogate and first principles model equations

# flux equation (gas constant is defined as J/mol-K)

def rule_flux(pervap, t, p, i):
    return pervap.permeate.flow_mol_phase_comp[t, p, i] == (pervap.permeance[t, p, i] *
                                                            exp(-pervap.energy_activation[t, p, i] /
                                                                (Constants.gas_constant *
                                                                 pervap.inlet.temperature[t]))
                                                           ) * pervap.area[t]
m.fs.pervap.eq_flux = Constraint(m.fs.time, m.fs.pervap.phase_list, m.fs.pervap.comp_list, rule=rule_flux)

# permeate condensation equation
# heat duty based on condensing all of permeate product vapor
# avoids the need for a Heater or HeatExchanger unit model

def rule_cond(pervap, t):
    return pervap.cond_duty[t] == sum(pervap.cond_heat[t, p, i] * pervap.permeate.flow_mol_phase_comp[t, p, i]
                                      for p in pervap.phase_list for i in pervap.comp_list)
m.fs.pervap.eq_cond = Constraint(m.fs.time, rule=rule_cond) 

# fix permeate vacuum pressure
m.fs.pervap.permeate.pressure.fix(1.3e3)

# flow equation adding total recovery as a custom mass balance term
def rule_flow(pervap, t, p, i):
    return pervap.retentate.flow_mol_phase_comp[t, p, i] == (pervap.inlet.flow_mol_phase_comp[t, p, i] -
                                                            pervap.permeate.flow_mol_phase_comp[t, p, i])
m.fs.pervap.eq_flow = Constraint(m.fs.time, m.fs.pervap.phase_list, m.fs.pervap.comp_list, rule=rule_flow)

Finally, let's define the Arc connecting our two models (IDAES Mixer and custom Pervaporation) and build the flowsheet network:

In [5]:
m.fs.s03 = Arc(source=m.fs.M101.outlet, destination=m.fs.pervap.inlet)
TransformationFactory("network.expand_arcs").apply_to(m)

Let's see how many degrees of freedom the flowsheet has:

In [6]:
print(degrees_of_freedom(m))

8


## 2.3 Inlet Specifications

By specifying inlet water, ethylene glycol, temperature and pressure for each feed stream, we obtain a square problem and can initialize and solve the flowsheet:

In [7]:
m.fs.M101.water_feed.flow_mol_phase_comp[0, "Liq", "water"].fix(0.34)  # mol/s
m.fs.M101.water_feed.flow_mol_phase_comp[0, "Liq", "ethylene_glycol"].fix(1e-6)  # mol/s
m.fs.M101.water_feed.temperature.fix(318.15)  # K
m.fs.M101.water_feed.pressure.fix(101.325e3)  # Pa

m.fs.M101.glycol_feed.flow_mol_phase_comp[0, "Liq", "water"].fix(1e-6)  # mol/s
m.fs.M101.glycol_feed.flow_mol_phase_comp[0, "Liq", "ethylene_glycol"].fix(0.66)  # mol/s
m.fs.M101.glycol_feed.temperature.fix(318.15)  # K
m.fs.M101.glycol_feed.pressure.fix(101.325e3)  # Pa

assert degrees_of_freedom(m) == 0

## 2.4 Custom Initialization
In addition to allowing custom variable and constraint defintions, the Skeleton model enables implementation of a custom initialization scheme. Complex unit operations may present unique tractability issues, and users have precise control over piecewise unit model solving.

In [8]:
def my_initialize(unit, **kwargs):
    # Callback for user provided initialization sequence
    unit.eq_flux.deactivate()
    unit.eq_cond.deactivate()
    unit.eq_flow.deactivate()
    assert degrees_of_freedom(unit) == 0
    if degrees_of_freedom(unit) == 0:
        res = solver.solve(m)
    unit.eq_flux.activate()
    unit.eq_cond.activate()
    unit.eq_flow.activate()
    print('Custom initiazation routine complete: ', res.solver.message)

solver = get_solver()
m.fs.M101.initialize()
propagate_state(m.fs.s03)
m.fs.pervap.config.initializer = my_initialize
my_initialize(m.fs.pervap)

results = solver.solve(m, tee = True)

2022-05-16 09:05:25 [INFO] idaes.init.fs.M101.water_feed_state: Starting initialization
2022-05-16 09:05:25 [INFO] idaes.init.fs.M101.water_feed_state: Property initialization: optimal - Optimal Solution Found.
2022-05-16 09:05:25 [INFO] idaes.init.fs.M101.glycol_feed_state: Starting initialization
2022-05-16 09:05:26 [INFO] idaes.init.fs.M101.glycol_feed_state: Property initialization: optimal - Optimal Solution Found.
2022-05-16 09:05:26 [INFO] idaes.init.fs.M101.mixed_state: Starting initialization
2022-05-16 09:05:26 [INFO] idaes.init.fs.M101.mixed_state: Property initialization: optimal - Optimal Solution Found.
2022-05-16 09:05:26 [INFO] idaes.init.fs.M101.mixed_state: Property package initialization: optimal - Optimal Solution Found.
2022-05-16 09:05:26 [INFO] idaes.init.fs.M101: Initialization Complete: optimal - Optimal Solution Found
Custom initiazation routine complete:  Ipopt 3.13.2\x3a Optimal Solution Found
Ipopt 3.13.2: nlp_scaling_method=gradient-based
tol=1e-06


*****

Let's check the results:

In [9]:
# print results
m.fs.M101.report()

print('Question for reviewers: Is there a way to add report() method with a stream table?')
m.fs.pervap.report()

print('Permeate water flow: ', value(m.fs.pervap.permeate.flow_mol_phase_comp[0, "Liq", "water"]), ' mol/s')
print('Permeate ethylene glycol flow: ', value(m.fs.pervap.permeate.flow_mol_phase_comp[0, "Liq", "ethylene_glycol"]), ' mol/s')
print('Permeate temperature: ', value(m.fs.pervap.permeate.temperature[0]), ' K')
print('Permeate pressure: ', value(m.fs.pervap.permeate.pressure[0]), ' Pa')
print('Retentate water flow: ', value(m.fs.pervap.retentate.flow_mol_phase_comp[0, "Liq", "water"]), ' mol/s')
print('Retentate ethylene glycol flow: ', value(m.fs.pervap.retentate.flow_mol_phase_comp[0, "Liq", "ethylene_glycol"]), ' mol/s')
print('Retentate temperature: ', value(m.fs.pervap.retentate.temperature[0]), ' K')
print('Retentate pressure: ', value(m.fs.pervap.retentate.pressure[0]), ' Pa')

# separation factor for results analysis - x is inlet water fraction, y is permeate water fraction
m.fs.x = Expression(expr=(m.fs.pervap.inlet.flow_mol_phase_comp[0, "Liq", "water"] /
                           sum(m.fs.pervap.inlet.flow_mol_phase_comp[0, "Liq", i] for i in m.fs.pervap.comp_list)
                          ))
m.fs.y = Expression(expr=(m.fs.pervap.permeate.flow_mol_phase_comp[0, "Liq", "water"] /
                           sum(m.fs.pervap.permeate.flow_mol_phase_comp[0, "Liq", i] for i in m.fs.pervap.comp_list)
                          ))
m.fs.separation_factor = Expression(expr=(m.fs.y/(1-m.fs.y)) / (m.fs.x/(1-m.fs.x)))

print('Inlet water mole fraction: ', value(m.fs.x))
print('Permeate water mole fraction: ', value(m.fs.y))
print('Separation factor: ', value(m.fs.separation_factor))
print('Condensation duty: ', value(m.fs.pervap.cond_duty[0]/1000), 'kW')
print('Duty per mole water recovered: ', value(
    m.fs.pervap.cond_duty[0]/(1000*m.fs.pervap.permeate.flow_mol_phase_comp[0, "Liq", "water"]*3600)),' kW-h / mol')


Unit : fs.M101                                                             Time: 0.0
------------------------------------------------------------------------------------
    Stream Table
                                                  Units       water_feed  glycol_feed   Outlet  
    Molar Flowrate ('Liq', 'water')            mole / second     0.34000  1.0000e-06     0.34000
    Molar Flowrate ('Liq', 'ethylene_glycol')  mole / second  1.0000e-06     0.66000     0.66000
    Temperature                                       kelvin      318.15      318.15      318.15
    Pressure                                          pascal  1.0132e+05  1.0132e+05  1.0132e+05
Question for reviewers: Is there a way to add report() method with a stream table?

Unit : fs.pervap                                                           Time: 0.0
Permeate water flow:  0.14258566520540567  mol/s
Permeate ethylene glycol flow:  0.0002667487709911813  mol/s
Permeate temperature:  298.15  K
Permeate pressu

In [10]:
# check results
assert check_optimal_termination(results)
assert_units_consistent(m)

assert value(m.fs.pervap.permeate.flow_mol_phase_comp[0, "Liq", "water"]) == pytest.approx(0.1426, rel=1e-3)
assert value(m.fs.pervap.permeate.flow_mol_phase_comp[0, "Liq", "ethylene_glycol"]) == pytest.approx(0.0002667, rel=1e-3)
assert value(m.fs.pervap.retentate.flow_mol_phase_comp[0, "Liq", "water"]) == pytest.approx(0.1974, rel=1e-3)
assert value(m.fs.pervap.retentate.flow_mol_phase_comp[0, "Liq", "ethylene_glycol"]) == pytest.approx(0.6597, rel=1e-3)
assert value(m.fs.separation_factor) == pytest.approx(1038, rel=1e-3)
assert value(m.fs.pervap.cond_duty[0]) == pytest.approx(5813, rel=1e-3)

# 3. Optimization

Suppose we wish to characterize the membrane behavior by calculating the maximum inlet water mole fraction allowing a separation factor of at least 100 (typical value for high-efficiency separation processes such as gas separation of CO2/N2). We need to add some lower bounds on the flow variables to ensure physically-sound solutions; alternatively, we could have specified these bounds while defining the variables above. We can quickly modify and resolve the model, and check some key results:

In [11]:
# unfix inlet flows but fix total to prevent divergence during solve
m.fs.M101.water_feed.flow_mol_phase_comp[0, "Liq", "water"].unfix()
m.fs.M101.glycol_feed.flow_mol_phase_comp[0, "Liq", "ethylene_glycol"].unfix()
m.fs.total_flow = Constraint(expr=m.fs.M101.water_feed.flow_mol_phase_comp[0, "Liq", "water"] +
                             m.fs.M101.glycol_feed.flow_mol_phase_comp[0, "Liq", "ethylene_glycol"] ==
                             1*pyunits.mol/pyunits.s)

# set criteria for separation factor
m.fs.sep_min = Constraint(expr=m.fs.separation_factor >= 100)

# set objective - defaults to minimization
m.fs.obj = Objective(expr=m.fs.x, sense=maximize)

results = solver.solve(m, tee = True)

Ipopt 3.13.2: nlp_scaling_method=gradient-based
tol=1e-06


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale

In [12]:
# print results
m.fs.M101.report()

print('Question for reviewers: Is there a way to add report() method with a stream table?')
m.fs.pervap.report()

print('Permeate water flow: ', value(m.fs.pervap.permeate.flow_mol_phase_comp[0, "Liq", "water"]), ' mol/s')
print('Permeate ethylene glycol flow: ', value(m.fs.pervap.permeate.flow_mol_phase_comp[0, "Liq", "ethylene_glycol"]), ' mol/s')
print('Permeate temperature: ', value(m.fs.pervap.permeate.temperature[0]), ' K')
print('Permeate pressure: ', value(m.fs.pervap.permeate.pressure[0]), ' Pa')
print('Retentate water flow: ', value(m.fs.pervap.retentate.flow_mol_phase_comp[0, "Liq", "water"]), ' mol/s')
print('Retentate ethylene glycol flow: ', value(m.fs.pervap.retentate.flow_mol_phase_comp[0, "Liq", "ethylene_glycol"]), ' mol/s')
print('Retentate temperature: ', value(m.fs.pervap.retentate.temperature[0]), ' K')
print('Retentate pressure: ', value(m.fs.pervap.retentate.pressure[0]), ' Pa')

print('Inlet water mole fraction: ', value(m.fs.x))
print('Permeate water mole fraction: ', value(m.fs.y))
print('Separation factor: ', value(m.fs.separation_factor))
print('Condensation duty: ', value(m.fs.pervap.cond_duty[0]/1000), 'kW')
print('Duty per mole water recovered: ', value(
    m.fs.pervap.cond_duty[0]/(1000*m.fs.pervap.permeate.flow_mol_phase_comp[0, "Liq", "water"]*3600)),' kW-h / mol')


Unit : fs.M101                                                             Time: 0.0
------------------------------------------------------------------------------------
    Stream Table
                                                  Units       water_feed  glycol_feed   Outlet  
    Molar Flowrate ('Liq', 'water')            mole / second     0.84240  1.0000e-06     0.84241
    Molar Flowrate ('Liq', 'ethylene_glycol')  mole / second  1.0000e-06     0.15760     0.15760
    Temperature                                       kelvin      318.15      318.15      318.15
    Pressure                                          pascal  1.0132e+05  1.0132e+05  1.0132e+05
Question for reviewers: Is there a way to add report() method with a stream table?

Unit : fs.pervap                                                           Time: 0.0
Permeate water flow:  0.14258566492542565  mol/s
Permeate ethylene glycol flow:  0.0002667487704468539  mol/s
Permeate temperature:  298.15  K
Permeate pressu

In [13]:
# check results
assert check_optimal_termination(results)
assert_units_consistent(m)

assert value(m.fs.pervap.permeate.flow_mol_phase_comp[0, "Liq", "water"]) == pytest.approx(0.1426, rel=1e-3)
assert value(m.fs.pervap.permeate.flow_mol_phase_comp[0, "Liq", "ethylene_glycol"]) == pytest.approx(0.0002667, rel=1e-3)
assert value(m.fs.pervap.retentate.flow_mol_phase_comp[0, "Liq", "water"]) == pytest.approx(0.6998, rel=1e-3)
assert value(m.fs.pervap.retentate.flow_mol_phase_comp[0, "Liq", "ethylene_glycol"]) == pytest.approx(0.1573, rel=1e-3)
assert value(m.fs.separation_factor) == pytest.approx(100.0, rel=1e-3)
assert value(m.fs.pervap.cond_duty[0]) == pytest.approx(5813, rel=1e-3)

# 4. Summary

The IDAES Skeleton Unit Model is a powerful tool for implementing relatively simple first-princples, surrogate-based or empirical unit operations. More crucially, users can add their own custom models and integrate them into a larger IDAES flowsheet without adding control volumes or rigorous flow balance and equilibrium calculations when not required. The pervaporation example displays a case where all model equations are empirical correlations or simple manual calculations, with a small number of state variable and port connections, and the Skeleton model avoids complex calculations that impact model tractability. The example also demonstrates adding a custom initialization scheme to handle internally model degrees of freedom, a feature providing greater user control than with most IDAES unit models.